In [84]:
#1. given four coordinates and a rotation angle, get XatTopEdge and YatLeftEdge.
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')
tl, 
#2. add x to top edge and y at left edge to get the top-left chiroptera point.

#3. calculate the reset between the chiroptera and atl, and add it to atl. 

#4. add the rotated offset to the atl.

#5. run correlation!


In [38]:
import numpy as np
1/np.cos(np.deg2rad(17.9)) * 1000

1050.867886489478

In [45]:
import math

import pandas as pd
import geopandas as gpd
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')

atl_df = pd.read_csv('unshifted_gt3r_full.csv')

b1 = 0
b2 = b1 + 4
chirop_df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')

x_min = min(chirop_df['x'])
x_max = max(chirop_df['x'])
y_min = min(chirop_df['y'])
y_max = max(chirop_df['y'])

# print(f'{x_min} {x_max} {y_min} {y_max}')
# box = np.array([[x_min, y_min], [x_min, y_max], [x_max, y_min], [x_max, y_max]])
# np.savetxt('box.txt', box)

#atl_df = atl_df[(atl_df['x'] >= x_min) & (atl_df['x'] <= x_max)]


#we are comparing the 1000:-1000 section of the (straight) ATL against the chiroptera in feature matching
#we should have the full match though.
#so we do 1/cos(17.9) * 1000 to get the y cutoffs

#this was primarily for comparison purposes
atl_df = atl_df[(atl_df['y'] >= y_min + 1051) & (atl_df['y'] <= y_max - 1051)]

#the original df WAS super long...
# atl_df = atl_df[(atl_df['y'] >= y_min) & (atl_df['y'] <= y_max)]

print(f'{x_min} {x_max} {y_min} {y_max}')

TL_POINT = (570762.9061543589, 9497299.94695935)

x_max = max(atl_df['x'])
y_max = max(atl_df['y'])

offset = (TL_POINT[0] - x_max, TL_POINT[1] - y_max)

atl_df['x'] = atl_df['x'] + offset[0]
atl_df['y'] = atl_df['y'] + offset[1]
atl_df.to_csv('debug_shifted_atl.csv', index=False)

569358.0299999999 571119.9599999998 9492739.940000001 9497243.080000002


In [46]:
import math
import numpy as np
ANGLE = 17.9
def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

best_fit = (34, 0)
best_fit = rotate((0,0), best_fit, np.deg2rad(ANGLE))
print(best_fit)
atl_df['x'] = atl_df['x'] + best_fit[0]
atl_df['y'] = atl_df['y'] - best_fit[1]

(32.354209731900895, 10.45012500519344)


In [47]:
atl_df.to_csv('debug_shifted_atl.csv', index=False)

In [ ]:
#if we wanted to align it, we could set the atl first point to the (x,y) of the first chiroptera.
#but we also want to get the right position..

#this uses the formula for rotation derived from trig addition formulas.
#set atl_df to left edge of the chiroptera
#read chiroptera[0]
import numpy as np



chirop_bl = chirop_df.iloc[0,2]
atl_bl = atl_df.iloc[0,0]

offset = chirop_bl - atl_bl
print(chirop_bl)
print(atl_bl)
print(offset)

atl_df['x'] = atl_df['x'] - offset

# ANGLE = -17.9
# #apply rotated shift
# best_fit = (35, 28)
# best_fit = rotate((0,0), best_fit, np.deg2rad(ANGLE))

#image coordinates are inverted.

# atl_df['x'] = atl_df['x'] + best_fit[0]
# atl_df['y'] = atl_df['y'] + best_fit[1]

#chirop_df.to_csv('debug_chirop_0_4.csv', index=False)
atl_df.to_csv('debug_atl_0_4.csv', index=False)